In [1]:
#setup
import numpy as np
import matplotlib.pyplot as plt
import random

import pandas as pd
import scipy.stats as sstats
from sklearn.model_selection import train_test_split

In [2]:
def KNN(train_X, train_Y, test_X, ks, verbose=False):
    """
    Compute predictions for various k
    Args:
        train_X: array of shape Ntrain x D
        train_Y: array of shape Ntrain
        test_X: array of shape Ntest x D
        ks: list of integers
    Returns:
        preds: dict k: predictions for k
    """
    # Cats data to float32
    train_X = train_X.astype(np.float32)
    test_X = test_X.astype(np.float32)

    # Alloc space for results
    preds = {}
    

    if verbose:
        print("Computing distances... ", end='')
    dists = -2 * np.dot(train_X, test_X.T) + np.sum(test_X**2, axis=1) + np.sum(train_X**2, axis=1)[:, np.newaxis]
    
    if verbose:
        print("Sorting... ", end='')

    closest = np.argsort(dists, axis=0)

    if verbose:
        print("Computing predictions...", end='')
    
    targets = train_Y[closest]

    for k in ks:
        predictions = sstats.mode(targets[:k])[0]
        predictions = predictions.ravel()
        preds[k] = predictions
    if verbose:
        print("Done")
    return preds

## Zadanie 1.
a) Napisz własny klasyfikator K-Nearest Neighbours (KNN). Zadanie można wykonać w Matlabie, Pythonie lub innym języku programowania. W zadaniu można użyć wcześniej zaimplementowanej funkcji liczenia odległości (na przykład z zadania 0 z listy 3).

b) Napisany klasyfikator KNN przetestuj na danych IRIS. Podziel dane losowo na dwie części: 100 wektorów danych użyj jako dane uczące do stworzenia klasyfikatora i 50 wektorów danych użyj jako dane testowe do przetestowania stworzonego klasyfikatora. Powtórz ten eksperyment kilkukrotnie i porównaj wyniki.

c) Przeprowadź też podobny test klasyfikatora KNN na danych Optical Recognition of Handwritten Digits (https://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits).

d) Zrób cross validation klasyfikatora KNN na obu zestawach danych: podziel dane na 10 części, kolejno bierz jedną z nich, traktuj ją jako dane testowe, a pozostałe 9 części jako dane uczące, stwórz klasyfikator na danych uczących i przetestuj na danych testowych, odnotuj liczbę błędów, powtórz obliczenia dla kolejnych części danych, policz całkowity błąd klasyfikatora sumując odnotowane liczby błędów.

In [16]:
#load iris
iris_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
iris_df = pd.read_csv(iris_url)

# Set the column names
iris_df.columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'target']

In [17]:
ks = np.arange(1, 10, 2)
for _ in range(5):
    train_iris_df, test_iris_df = train_test_split(iris_df, test_size=0.3333)

    train_x = np.array(train_iris_df.drop(['target'], axis=1))
    train_y = np.array(train_iris_df['target'])

    test_x = np.array(test_iris_df.drop(['target'], axis=1))
    test_y = np.array(test_iris_df['target'])

    err_rate = {}
    knn = KNN(train_x, train_y, test_x, ks)
    for k, clas in knn.items():
        err_rate[k] = np.count_nonzero(clas != test_y) / len(clas)

    print('Error rates: ', err_rate)

Error rates:  {1: 0.04, 3: 0.02, 5: 0.0, 7: 0.02, 9: 0.02}
Error rates:  {1: 0.04, 3: 0.04, 5: 0.04, 7: 0.04, 9: 0.04}
Error rates:  {1: 0.06, 3: 0.04, 5: 0.04, 7: 0.0, 9: 0.02}
Error rates:  {1: 0.04, 3: 0.02, 5: 0.02, 7: 0.02, 9: 0.02}
Error rates:  {1: 0.04, 3: 0.04, 5: 0.02, 7: 0.0, 9: 0.0}


In [13]:
#load Optical Recognition
digits_train_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/optdigits/optdigits.tra'
digits_test_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/optdigits/optdigits.tes'
digits_train_df = pd.read_csv(digits_train_url, header=None)
digits_test_df = pd.read_csv(digits_test_url, header=None)

train_x = np.array(digits_train_df)[:, :-1]
train_y = np.array(digits_train_df)[:, -1]

test_x = np.array(digits_test_df)[:, :-1]
test_y = np.array(digits_test_df)[:, -1]

In [15]:
ks = np.arange(1, 10, 2)
err_rate = {}
knn = KNN(train_x, train_y, test_x, ks)
for k,  clas in knn.items():
    err_rate[k] = np.count_nonzero(clas != test_y) / len(clas)

print('Error rates: ', err_rate)

Error rates:  {1: 0.02003338898163606, 3: 0.021702838063439065, 5: 0.021146355036171398, 7: 0.02337228714524207, 9: 0.022259321090706732}
